In [ ]:
%cd /content/drive/MyDrive/amnlp_project_round_2/mnli_sota

/content/drive/MyDrive/amnlp_project_round_2/mnli_sota


In [ ]:
# pip 
!pip install transformers
!pip install datasets

     |████████████████████████████████| 2.6 MB 4.0 MB/s 
     |████████████████████████████████| 636 kB 74.1 MB/s 
     |████████████████████████████████| 895 kB 67.0 MB/s 
     |████████████████████████████████| 3.3 MB 24.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 4.1 MB/s 
     |████████████████████████████████| 118 kB 79.1 MB/s 
     |████████████████████████████████| 243 kB 54.5 MB/s 


In [ ]:
# imports 

import torch
import numpy as np
import transformers
from datasets import load_dataset, load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [ ]:
task = "mnli"
model_checkpoint = "bert-base-uncased"
batch_size = 16

dataset = load_dataset("glue", task)
metric = load_metric('glue', task)

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mnli/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


Downloading:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
  return tokenizer(examples["premise"], examples["hypothesis"], truncation=True)

In [ ]:
encoded_dataset = dataset.map(preprocess_function, batched=True)

  0%|          | 0/393 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

watermarked_checkpoint_chars = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/seed_3/checkpoint_watermarked_chars.pkl")['bert']
# watermarked_checkpoint_semantics = torch.load("/content/drive/MyDrive/amnlp_project_round_2/checkpoints/checkpoint_watermarked_semantics.pkl")['bert']

model.bert.load_state_dict(watermarked_checkpoint_chars)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [ ]:
metric_name = "accuracy"

args = TrainingArguments(
    "mnli",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
)

In [ ]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  predictions = np.argmax(predictions, axis=1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
validation_key = "validation_matched"
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset[validation_key],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: premise, idx, hypothesis.
***** Running training *****
  Num examples = 392702
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 122720


Epoch,Training Loss,Validation Loss,Accuracy
1,0.463500,0.450552,0.828935
2,0.353000,0.471363,0.839633
3,0.250500,0.532029,0.836373
4,0.180900,0.731732,0.841875
5,0.128400,0.850200,0.844320


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: premise, idx, hypothesis.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to mnli/checkpoint-24544
Configuration saved in mnli/checkpoint-24544/config.json
Model weights saved in mnli/checkpoint-24544/pytorch_model.bin
tokenizer config file saved in mnli/checkpoint-24544/tokenizer_config.json
Special tokens file saved in mnli/checkpoint-24544/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: premise, idx, hypothesis.
***** Running Evaluation *****
  Num examples = 9815
  Batch size = 16
Saving model checkpoint to mnli/checkpoint-49088
Configuration saved in mnli/checkpoint-49088/config.json
Model weights saved in mnli/checkpoint-49088/pytorch_model.bin
tokenizer config file sa

TrainOutput(global_step=122720, training_loss=0.28565309139398426, metrics={'train_runtime': 19146.0443, 'train_samples_per_second': 102.554, 'train_steps_per_second': 6.41, 'total_flos': 8.18623276008416e+16, 'train_loss': 0.28565309139398426, 'epoch': 5.0})

In [ ]:
# save finetuned model
torch.save(model.bert, f="/content/drive/MyDrive/amnlp_project_round_2/mnli_sota/ft_checkpoints/after_ft_seed_3/after_ft_seed_3_checkpoint.pkl")